
### <center> Автор материала: Юлия Климушина

## <center> Прогноз цен на подержанные автомобили </center>
### <center> Индивидуальный проект по анализу данных

В этом проекте мы будем решать задачу восстановления регрессии. Данные, используемые в этом проекте, можно скачать [тут](https://www.kaggle.com/orgesleka/used-cars-database). Датасет содержит информацию о подержанных автомобилях, выставленных на продажу на Ebay в марте-апреле 2016. Данные представлены на немецком языке. 
Цель исследования: создание модели, предсказывающую цену автомобиля на вторичном рынке. Такая модель может помочь: 
- владельцу авто, желающему продать своего железого коня, не продешевить;
- покупателю не переплатить;
- диллеру, занимающемуся перепродажей машин, определить насколько выгодно конкретное предложение, за какую цену  можно перепродать автомобиль.

###  1. Описание набора данных и признаков

#### Список признаков:
- **dateCrawled**: дата и время первого просмотра объявления 
- **name** : название машины (сформировано из названия марки, модели и другой информации)
- **seller** : кто продает ('privat' - частное лицо, 'gewerblich' - диллер)
- **offerType** : тип предложения ('Angebot' - продажа, 'Gesuch' - покупка)
- **price** : цена
- **abtest** : A/B тест. Покопавшись в интернете, я выяснила, что A/B тестирование - это тактика, с помощью которой макетологи выясняют, какие заголовки объявления, текст, изображения, призывы к действию будут лучше работать для целевой аудитории.
- **vehicleType** : тип авто ('coupe', 'suv', 'kleinwagen', 'limousine', 'cabrio', 'bus', 'kombi', 'andere' - "купе", "внедорожник", "миниавто", "седан", "кабриолет", "автобус", "комби", "другое", соответственно)
- **yearOfRegistration** : в каком году машина была впервые зарегистрирована
- **gearbox** : тип коробки передач ('manuell' - ручная, 'automatik' - автоматическая)
- **powerPS** : мощность
- **model** : модель
- **kilometer** : пробег в километрах
- **monthOfRegistration** : в каком месяце машина была впервые зарегистрирована
- **fuelType** : тип топлива
- **brand** : марка
- **notRepairedDamage** : есть ли повреждения, требующие ремонта ('ja' - да, 'nein' - нет)
- **dateCreated** : дата и время создания объявления на eBay
- **nrOfPictures** : количество фотографий автомобиля (к сожалению, это поле везде содержит нули и поэтому бесполезно)
- **postalCode** : почтовый индекс
- **lastSeenOnline** : дата и время последнего просмотра объявления

Целевая переменная: **price** - цена автомобиля. Перед нами стоит задача восстановления регрессии.

###  2. Первичный анализ и обработка признаков

Посмотрим на данные

Давайте избавимся от пропусков и не информативных признаков.
**abtest** явно лишний признак, так как не имеет отношения к автомобилям как таковым. Насколько мне удалось выяснить, это некий показатель, используемый Ebay для определения эффективности рекламы. 

Меня интересуют только объявления о продаже, поэтому я удалю строки о покупке и признак **offerType**.<br>
Признак **vehicleType** содержит пропуски. Можно заменить их значением, обозначающим "другое" (andere)<br>
Строки с пропусками в **gearbox** удалим.

Давайте посмотрим на признак **brand**. 

"sonstige_autos" означает "прочие автомобили". Строк с такими значения немного и анализ показывает, что это старые и/или редкие машины, информации по которым не достаточно, чтобы строить прогноз, к тому же поле **model** у них не заполнено, поэтому удалим такие строки.

В признаке **model** 13433 пропусков, зато **brand** всегда заполнен. Учитывая, что **name** часто содержит в себе информацию о марке и модели, достанем модель оттуда. Строки, которые не подойдут под алгоритм, удалим. После этой операции удалим переменную **name**, он нам больше не пригодится.

Разберемся с топливом **fuelType**. Заменим пустующие значения 'andere' ('другое'). 

Признак **notRepairedDamage** имеет 56335 пропусков. Можно исходить из предположения, что если владелец не упомянул в объявлении про повреждения, то он продает её как не требующую ремонта. Пометим такие пропуски как 'nein' и приведем к бинарному формату.

Посмотрим на статистику.

Основные выводы:
**nrOfPictures** по нулям. Удаляем этот признак.
в **price** наблюдаются большие выбросы (10 в восьмой многовато даже для Bloodhound SSC)
**kilometer** - имеет скошенное влево распределение

###  3. Визуальный анализ признаков. Особенности данных.

Посмотрим на разброс значений количественных признаков: цены, года первой регистации и мощности.

Из-за выбросов ничего не разобрать. Начнем с цен, определим пороги отсечения выбросов. Просмотр объявлений о продаже показали, что старая машина (15-20 лет) может стоить в районе 100 евро. В качестве верхней границы возьмем 150000. Именно столько стоят Porsche, которых не так уж мало в наборе.   

Посмотрим на год первой регистрации. Это важный признак, т.к. возраст автомобиля один из ключевых факторов, влияющих на его цену. Очевидно, что год регистрации не может быть позднее, чем год размешения объявлений. Объявления размещены в марте и апреле 2016, поэтому 2016-й год также не будем рассматривать. 17141 автомобилей зарегистрированы в 2016 и позднее. Удалим эти строки и машины старее 1976 года, то есть оставляем период в 40 лет.

В выборке есть достаточно автомобилей марки Porsche, мощность двигателей которых может превышать 500 л.с. Также есть авто марки Fiat с мощность движка не превышающим 30 лошадок. Возьмем ннтервал допустимых значений (20, 600). И построим ящики с усами.

Признак **monthOfRegistration** целочисленный, но для модели не имеет смысла сравнивать январь с сентябрем, поэтому переквалифицируем его в категориальный. Месяц 0 будем рассматривать, как "не определено". Как видно из гистограммы с марта по июль публикуется больше объявлений о продаже авто, чем в остальные месяцы.

Посмотрим на распределение количественных и бинарных признаков.

Обратим внимание на пробег. Медиана и максимум = 150000. 65% машин имеют пробег 150000 км и мы имеем скошенное распределение (длинный левый хвост). У распределений целевой переменной **price** и мощности **powerPS** видим длинный правый хвост. Попробуем преобразовать данные, с тем, чтобы приблизить их распределения к нормальному. Для этого пробег возведем в степень, а **price** и **powerPS** - логарифмируем.

Изобразим матрицу корреляции

**Price** положительно коррелирует с **yearOfRegistration** и **powerPS** и отрицательно - с **kilometer**. 

Посмотрим на некоторые категориальные признаки по отдельности.

Оказывается у нас только одна запись с **seller** == 'gewerblich', так что можно удалить этот признак. fuelType отличный от diesel и benzin объединим в общую группу andere.

Посмотрим на взаимодействие категориальных признаков и цены.

Автомобили с дизельными двигателями дороже бензиновых и прочих, а машины с автоматической коробкой дороже, чем с ручной. 

Из этой картинки следует, что Porsche существенно дороже остальных марок.

Признак **postalCode** мы исключим. Если бы рассматривался российский или, к примеру, американский рынок, то имело бы смысл поработать с ним, так как цены от области к области (от штата к штату) варьируются. На Камчатке и Аляске они, вероятно, выше, чем по стране. Но поскольку речь идет о Германии, то отбросим этот признак.

###  Предобработка данных 

Разделим данные на тренировочную и тестовую части и применим dummy-кодирование к категориальным признакам. 

###  Построение базовых моделей. Выбор метрик качества. 

Давайте построим и сравним линейную модель и случайный лес. 

#### Базовая модель линейной регрессии

Отмасштабируем признаки

В качестве метрик качества линейной регрессии выберем MAE за его интерпретируемость. Также посмотрим на коэффициент детерминации, или коэффициент $R^2$. Данная мера качества — это нормированная среднеквадратичная ошибка. Чем она ближе к единице, тем лучше модель объясняет данные.

#### Базовая модель случайного леса

Учитывая, что целевая переменная принимает значения в диапазоне (4.62, 11.51), то ошибка выглядит допустимой.

###  Создание новых признаков и описание этого процесса

Новые признаки, которые предположительно могут коррелировать с целевым: 
- **adUpDays** - сколько дней висело объявление   
- **kilPerYear** - среднегодовой пробег.

Применим к признакам со смещенным распределением логарифмирование.

Добавление новых признаков и полиномов второй степени улучшило качество линейной модели, но оно всё же хуже, чем у базового случайного леса, поэтому сосредоточимся на последнем. 

Модель случайного леса немного улучшилась при добавлении **adUpDays**. Добавление **kilPerYear** никак не повлияло на качество.

Какие же признаки оказались наиболее важными для модели случайного леса. Как видно из графика ниже, наиболее важным признаком оказался год регистрации, второй по важности признак - мощность двигателя, третий - **notRepairedDamage**.

###  Построение кривых валидации

Попробуем улучшить результат случайного леса. Для начала посмотрим как ведут себя кривые валидации при изменении основных параметров.

Начнем с количества деревьев:

Как видно при достижении 30 деревьев точность модели на тесте выходит на асимптоту. 

Давайте посмотрим какие параметры регуляризации добавить в модель, чтобы недопустить переобучение.
Посмотрим как ведет себя модель в зависимости от параметра максимальной глубины – `max_depth`.

Как видим, строить деревья глубиной более 22 смысла не имеет, качество на тесте выходит на ассиптоту.<br> 
<br>
Построим кривые валидации для параметра `min_samples_leaf`.

Как мы видим на тесте максимальное качество достигается, если минимальном числе объектов в листе 3. <br>
<br>
Параметр `max_features` определяет количество случайных признаков из `n`  исходных. Для задач регрессии рекомендуется использовать $\frac{n}{3}$. Давайте определим оптимальный параметр для нашего случая.

**max_features** = 200 - оптимальный вариант.

###  Кросс-валидация, подбор параметров

### Прогноз для тестовой выборки

Обучим случайный лес с оптимальными гиперпараметрами и получим прогноз для тестовой выборки. 

Нам удалось выиграть "аж" 0.01 на обоих метриках.

###  Оценка модели

Переведем целевой признак обратно в километры и посмотрим насколько хорошо наша модель отработала на тестовом наборе. Для начала сравним реальные и предсказанные цены по нескольким примерам.

Как видим модель довольно сильно ошибается в некоторых случаях, но в целом закономерности в данных выявлены и результат выглядит неплохо. На графике ниже видим рассеяние реальной цены vs. предсказанной цены относительно линии идентичности (красная линия). Хорошо видно, что чем больше цена, тем сильнее ошибается модель и видно тенденцию к недооценке. Очевидно это связано с недостаточностью данных по дорогим автомобилям.  

Также видим в самом начале какие-то сильные выбросы. Посмотрев на данные, становится ясно, что модель предсказывает более высокую стоимость машин, выставленных за "бесценок". Возможно продавцы указывают низкую цену, чтобы привлечь покупателей, а может забыли указать, что машина требует ремонта. 

###  Построение кривых обучения

Давайте построим кривые обучения для всего объема данных.

Большое расстояние между кривыми указывает на переобучение. 

### Выводы 

Сырые данные содержали много пропусков и выбросы. Нам потребовалось провести значительную обработку и фильтрацию. К категориальным признакам мы применили one-hot encoding. Целевой признак имел сильно скошенное распределение, поэтому мы применили к нему логарифмическое преобразование.  

Мы сравнили две модели и пришли к выводу, что линейная модель с полиномиальными признаками дает MAE: 0.31 и $R^2$: 0.85, в то время как случайный лес "из коробки" сразу выдал MAE:  0.28 и $R^2$: 0.87. К сожалению, нам не удалось синтезировать признаки, улучшающие этот результат случайного леса. Настройка гиперпараметров привела к незначительному росту качества: MAE:  0.27 и $R^2$: 0.88. 

Учитывая, что случайный лес показал неплохие результаты, с моей стороны было бы упущением не попровать бустинг на наших данных. Я воспользовалась питоновской реализацией XGBoost, понастраивала параметры с помощью hyperopt, но значимых улучшений не получила, поэтому решила не докучать читателям и опустила выкладки. 

Таким образом можно заключить, что применительно к этой задаче случайный лес сработал хорошо.

Исходя из кривых обучения можно сделать вывод, чтобы улучшить модель можно:
- увеличить размер выборки;
- понизить сложность модели. Этого можно добиться, если использовать меньший набор признаков или увеличить регуляризацию (к примеру, можно снизить глубину деревьев или увеличить минимальном число объектов в листе).